In [4]:
from __future__ import division, print_function
import numpy as np
import codecs
import re
import os

%load_ext autoreload
%autoreload 2

In [5]:
from nltk.tokenize import word_tokenize

In [6]:
from collections import Counter

In [122]:
# text_file = '/home/chris/test/text8/text8'

# SPANISH NEWS COMMENTARY
nc_text_file = '/home/chris/projects/angular/editor_components/microservices/vocabulary_server/test_data/text_for_lm/news-commentary-v8.es.gz'

# SPANISH WIKIPEDIA DATA
wiki_text_file = '/home/chris/projects/angular/editor_components/microservices/vocabulary_server/proto/spanish_data/spanish_wikipedia_data.txt'

In [123]:
import gzip

In [124]:
# Utils for autocomplete
def extract_phrases(tokens, min_len=1, max_len=2):
    phrase_list = []
    for i in range(min_len, min(len(tokens)+1, max_len+1)):
        phrase_list.extend([tokens[j:j+i] for j in range(len(tokens)-i+1)])
    return phrase_list


In [132]:
# TODO: pad each line with _START_ and _END_ to get real sequence probs?
ngram_counts = Counter()
lm_order = 3
max_lines = 200000

with gzip.open(nc_text_file, 'rb') as input:
    pos = 0
    for line in list(input)[:max_lines]:
        if pos % 10000 == 0:
            print(pos)
        pos += 1
        line = line.decode('utf8').strip()
        tokens = word_tokenize(line, language='spanish')
#         print(tokens)
        # get all ngrams from two up to lm_order
        ngrams = extract_phrases(tokens, min_len=1, max_len=3)
#         print(ngrams)
        # embarrasingly parallel
        for ngram in ngrams:
#             print(type(ngram))
            # the dict syntax here is important -- otherwise each string will update individually (as if unigram)
            ngram_counts.update({tuple(ngram): 1})
            # if first item is uppercase, also add the lowercase version
            if ngram[0][0].isupper():
                lower_ngram = [w.lower() for w in ngram]
                ngram_counts.update({tuple(lower_ngram): 1})
                
        


0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000


In [133]:
# now do the same for spanish wikipedia data
with open(wiki_text_file, 'rb') as input:
    pos = 0
    for line in list(input)[:max_lines]:
        if pos % 10000 == 0:
            print(pos)
        pos += 1
        line = line.decode('utf8').strip()
        tokens = word_tokenize(line, language='spanish')
#         print(tokens)
        # get all ngrams from two up to lm_order
        ngrams = extract_phrases(tokens, min_len=1, max_len=3)
#         print(ngrams)
        # embarrasingly parallel
        for ngram in ngrams:
#             print(type(ngram))
            # the dict syntax here is important -- otherwise each string will update individually (as if unigram)
            ngram_counts.update({tuple(ngram): 1})
            
            # if first item is uppercase, also add the lowercase version
            if ngram[0][0].isupper():
                lower_ngram = [w.lower() for w in ngram]
                ngram_counts.update({tuple(lower_ngram): 1})
                


0


In [138]:
min_unigram_threshold = 25
min_count_threshold = 500
min_key_char_len = 3

In [139]:
len(ngram_counts.keys())

4446359

In [140]:
for key, value in ngram_counts.items():
    if len(key) == 1 and (value < min_unigram_threshold or len(' '.join(key)) < min_key_char_len):
        del ngram_counts[key]
    elif len(key) > 1 and (value < min_count_threshold or len(' '.join(key)) < min_key_char_len):
        del ngram_counts[key]
        

In [141]:
len(ngram_counts.keys())

14018

In [142]:
ngram_counts[(u'españa',)]

765

In [143]:
[i for i in ngram_counts.most_common()][:100]

[((u'que',), 149520),
 ((u'los',), 134138),
 ((u'las',), 73331),
 ((u'del',), 62400),
 ((u'una',), 55458),
 ((u'de', u'la'), 50677),
 ((u'para',), 48326),
 ((u'por',), 45929),
 ((u'con',), 38286),
 ((u'm\xe1s',), 35412),
 ((u'de', u'los'), 32208),
 ((u'como',), 25853),
 ((u'en', u'el'), 20281),
 ((u'pero',), 19914),
 ((u'sus',), 19213),
 ((u'a', u'la'), 16960),
 ((u'de', u'las'), 16763),
 ((u'en', u'la'), 16644),
 ((u',', u'la'), 16086),
 ((u',', u'el'), 15898),
 ((u'a', u'los'), 13885),
 ((u'pa\xedses',), 13648),
 ((u'estados',), 13393),
 ((u'Estados',), 12134),
 ((u'sin',), 11690),
 ((u'que', u'se'), 11440),
 ((u'lo', u'que'), 10992),
 ((u',', u'los'), 10946),
 ((u',', u'y'), 10795),
 ((u'unidos',), 10738),
 ((u'sobre',), 10673),
 ((u'Unidos',), 10662),
 ((u'estados', u'unidos'), 10572),
 ((u'Estados', u'Unidos'), 10570),
 ((u'son',), 10562),
 ((u'y', u'la'), 10526),
 ((u',', u'pero'), 10455),
 ((u'han',), 10429),
 ((u'de', u'que'), 10391),
 ((u'que', u'la'), 10145),
 ((u'Los',), 100

In [144]:
# map the tuple keys to strings (they're going to be queried as strings in the UI)
string_ngram_counts = dict()

In [145]:
for k,v in ngram_counts.items():
    string_ngram_counts[' '.join(k)] = v

In [146]:
# dump keys and counts to json
import json
with codecs.open('spanish.vocabulary.counts.json','w', encoding='utf8') as json_out:
    json_out.write(json.dumps(string_ngram_counts, indent=2))
    
# dump just keys to json (flat vocab list)
with codecs.open('spanish.vocabulary.list.json','w', encoding='utf8') as json_out:
    json_out.write(json.dumps([k for k in string_ngram_counts.keys()], indent=2))

In [125]:
num_big =0
for key in ngram_counts.keys():
    if len(ngram_counts[key].values()) < 20:
        del ngram_counts[key]
    elif len(key) == 3 and len(ngram_counts[key].values()) > 100:
        num_big += 1
        print(key)
num_big

(u'it', u'does', u'not')
(u'been', u'able', u'to')
(u'(', u'such', u'as')
(u'needs', u'to', u'be')
(u',', u'at', u'the')
(u'to', u'do', u'with')
(u'to', u'have', u'been')
(u'the', u'world', u'bank')
(u',', u'and', u'in')
(u'growth', u'in', u'the')
(u',', u'and', u'then')
(u'the', u'west', u',')
(u'because', u'they', u'are')
(u'of', u'course', u',')
(u',', u'to', u'be')
(u'that', u'they', u'were')
(u'the', u'case', u'of')
(u'part', u'of', u'the')
(u'the', u'bush', u'administration')
(u'owing', u'to', u'the')
(u'of', u'those', u'who')
(u'it', u'was', u'the')
(u'in', u'which', u'the')
(u',', u'we', u'are')
(u',', u'this', u'is')
(u',', u'the', u'current')
(u'to', u'the', u'us')
(u'has', u'become', u'a')
(u'same', u'time', u',')
(u'will', u'continue', u'to')
(u'of', u'the', u'us')
(u'is', u'that', u'the')
(u'more', u'than', u'the')
(u'that', u'have', u'been')
(u'a', u'lack', u'of')
(u',', u'which', u'has')
(u'a', u'group', u'of')
(u',', u'but', u'a')
(u',', u'however', u',')
(u',', u'is', 

505

In [126]:
len(ngram_counts.keys())


35673

In [127]:
from blist import sortedlist
def sorted_list_dd():
    # sort descending by score
    return sortedlist([], key=lambda i: -i[1])

In [128]:
ngram_scores = defaultdict(sorted_list_dd)

for key in ngram_counts:
    total_occs = sum(ngram_counts[key].values())
    # items should already be in descending frequency order
    for word, count in ngram_counts[key].items():
        probability_score = np.log(float(count) / total_occs)
        ngram_scores[key].add((word, probability_score))

In [130]:
ngram_scores[tuple([u'one', u'of', u'the'])]

sortedlist([(u'most', -1.7572474169229115), (u'world\u2019s', -2.4415839678007019), (u'main', -3.22653869761377), (u'few', -3.3276348144851386), (u'great', -3.7765850345330421), (u'first', -3.7765850345330421), (u'best', -3.881945550190868), (u'key', -3.9997285858472518), (u'world', -4.0422882002660474), (u'greatest', -4.1332599784717745), (u'two', -4.2333434370287568), (u'largest', -4.2333434370287568), (u'worst', -4.2874106582990326), (u'major', -4.2874106582990326), (u'reasons', -4.2874106582990326), (u'more', -4.3445690721389809), (u'highest', -4.4697322150929875), (u'many', -4.6128330587336608), (u'biggest', -4.6928757664071972), (u'poorest', -4.7798871433968264), (u'three', -4.7798871433968264), (u'oldest', -4.8751973232011512), (u'richest', -4.980557838858978), (u'lowest', -4.980557838858978), (u'country\u2019s', -5.0983408745153609), (u'central', -5.3860229469671426), (u'top', -5.3860229469671426), (u'least', -5.3860229469671426), (u'countries', -5.3860229469671426), (u'last', 

In [96]:
# map to list for pickling
ngram_lookup_obj = defaultdict(list)
for key, sorted_vals in ngram_scores.items():
    ngram_lookup_obj[key] = list(sorted_vals)

In [97]:
# WORKING:
# get the actual probability of each following token
# tokens should be scored by both length and probability -- only longer tokens will really save the user time
# However, the second prefix filtering step should let the user quickly find longer tokens
# (but still weighted by probability)

# we may want to filter punctuation out of the counts, at least when they are returned to the user

In [98]:
ngram_counts[tuple([u'a', u'very'])]


Counter({u'different': 59, u'large': 23, u'high': 22, u'small': 18, u'low': 18, u'good': 13, u'strong': 13, u'long': 11, u'real': 8, u'poor': 8, u'big': 7, u'few': 7, u'short': 7, u'important': 6, u'serious': 6, u'bad': 5, u'special': 4, u'expensive': 4, u'sharp': 4, u'significant': 4, u'mild': 3, u'uncomfortable': 3, u'complicated': 3, u'narrow': 3, u'simple': 3, u'powerful': 3, u'dangerous': 3, u'similar': 3, u'awkward': 3, u'difficult': 3, u'limited': 2, u'precarious': 2, u'valuable': 2, u'distant': 2, u'controversial': 2, u'complex': 2, u'attractive': 2, u'specific': 2, u'old': 2, u'hard': 2, u'broad': 2, u'conservative': 2, u'open': 2, u'successful': 2, u'modern': 2, u'delicate': 2, u'measured': 2, u'high-risk': 1, u'cheap': 1, u'cold': 1, u'high-level': 1, u'fine': 1, u'unhealthy': 1, u'profitable': 1, u'rapid': 1, u'internationally': 1, u'adverse': 1, u'young': 1, u'transparent': 1, u'easy': 1, u'selfish': 1, u'smart': 1, u'traditionalistic': 1, u'low-grade': 1, u'early': 1, u's

In [100]:
import cPickle
cPickle.dump(ngram_lookup_obj, open('europarl.test.ngram_scores.cpkl', 'wb'))

In [102]:
test_loaded = cPickle.load(open('europarl.test.ngram_scores.cpkl', 'rb'))

In [6]:
from collections import defaultdict
def rec_dd():
    return defaultdict(rec_dd)

In [ ]:
# Counter API
# sum(c.values())                 # total of all counts
# c.clear()                       # reset all counts
# list(c)                         # list unique elements
# set(c)                          # convert to a set
# dict(c)                         # convert to a regular dictionary
# c.items()                       # convert to a list of (elem, cnt) pairs
# Counter(dict(list_of_pairs))    # converttofrom a list of (elem, cnt) pairs
# c.most_common()[:-n-1:-1]       # n least common elements
# c += Counter()                  # remove zero and negative counts

In [1]:
>>> c = Counter(a=3, b=1)
>>> d = Counter(a=1, b=2)
>>> c + d                       # add two counters together:  c[x] + d[x]
Counter({'a': 4, 'b': 3})
>>> c - d                       # subtract (keeping only positive counts)
Counter({'a': 2})
>>> c & d                       # intersection:  min(c[x], d[x])
Counter({'a': 1, 'b': 1})
>>> c | d                       # union:  max(c[x], d[x])
Counter({'a': 3, 'b': 2})

NameError: name 'Counter' is not defined

In [75]:
x = Counter()
x.update({tuple(['t', 'x']): 1})

In [76]:
x.keys()

[('t', 'x')]